# Subspace-Search Variational Quantum Eigensolver (SSVQE)

Subspace-Search VQE (SSVQE) finds the lowest $k$ eigenpairs $\{(\lambda_j,\ket{\psi_j})\}_{j=0}^{k-1}$ of a Hamiltonian $\hat H$ in a **single variational optimization**. The key ideas and steps are:

1. **Choose orthogonal reference states**  
   Select $k$ mutually orthogonal basis states  
   \begin{equation*}
     \{\ket{\rho_0}, \ket{\rho_1}, \dots, \ket{\rho_{k-1}}\},
     \quad
     \braket{\rho_i | \rho_j} = \delta_{ij}.
   \end{equation*}

2. **Variational ansatz**  
   Define a parameterized unitary $U_V(\boldsymbol\theta)$.  Apply it to each reference to form ansatz states  
   \begin{equation*}
     \ket{\psi_j(\boldsymbol\theta)} = U_V(\boldsymbol\theta)\,\ket{\rho_j}, 
     \quad j=0,\dots,k-1.
   \end{equation*}

3. **Weighted cost function**  
   Assign strictly decreasing positive weights $w_0>w_1>\cdots>w_{k-1}$.  Define  
   \begin{equation*}
     C(\boldsymbol\theta)
     = \sum_{j=0}^{k-1} w_j \,\bra{\psi_j(\boldsymbol\theta)}\,\hat H\,\ket{\psi_j(\boldsymbol\theta)}.
   \end{equation*}
   The largest weight $w_0$ forces $\ket{\psi_0}$ toward the ground state, the next weight $w_1$ pushes $\ket{\psi_1}$ into the first excited subspace, and so on.

4. **Optimization**  
   Use a classical optimizer (COBYLA) to minimize $C(\boldsymbol\theta)$.  Since $U_V$ is unitary, the $\ket{\psi_j(\boldsymbol\theta)}$ remain orthogonal throughout, ensuring they span the lowest-energy subspace.

5. **Extract eigenpairs**  
   At convergence:
   \begin{equation*}
     \lambda_j \approx \bra{\psi_j(\boldsymbol\theta^*)}\,\hat H\,\ket{\psi_j(\boldsymbol\theta^*)},
     \quad
     \ket{\psi_j} = \ket{\psi_j(\boldsymbol\theta^*)},
     \quad j=0,\dots,k-1.
   \end{equation*}
   These approximate the true eigenvalues $\lambda_0<\lambda_1<\dots<\lambda_{k-1}$.


More informations can be found in the Qiskit course [Qiskit SSVQE](https://learning.quantum.ibm.com/course/variational-algorithm-design/instances-and-extensions#subspace-search-vqe-ssvqe) and in the original paper [Subspace-search variational quantum eigensolver for excited states](http://dx.doi.org/10.1103/PhysRevResearch.1.033062)

In [1]:
from qiskit import QuantumCircuit
import numpy as np
from scipy.optimize import minimize
from qiskit.primitives import StatevectorEstimator
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp, Statevector, Operator
from qiskit.circuit import Parameter

estimator = StatevectorEstimator()

### Reference State Preparation

The function `basis_state_circuit(index, num_qubits)` builds a circuit that prepares the computational basis state $\ket{\text{index}}$ on `num_qubits` qubits. It converts the integer `index` to its `num_qubits`-bit binary representation. It applies an X gate to each qubit whose bit is 1, leaving qubits with bit 0.

In [2]:
def basis_state_circuit(index, num_qubits):
    qc = QuantumCircuit(num_qubits)
    binary_string = format(index, f'0{num_qubits}b')
    for qubit, bit in enumerate(reversed(binary_string)):  # qubit 0 is rightmost
        if bit == '1':
            qc.x(qubit)
    return qc


### SSVQE Cost Function

This function builds $k$ circuits by preparing each reference basis state, appending the shared variational ansatz, and measuring $\langle H\rangle$. It then returns the weighted sum of these energies. Minimizing this cost drives each ansatz output toward a different eigenstate, with larger weights prioritizing the lower-energy states.  


In [3]:
def cost_func_ssvqe(params, basis_state_indices, hamiltonian, estimator, num_qubits, weights=None, shared_ansatz=None):
    # Build circuits
    circuits = []
    for idx in basis_state_indices:
        basis_circ = basis_state_circuit(idx, num_qubits)
        full_circ = basis_circ.compose(shared_ansatz)
        circuits.append((full_circ, hamiltonian, [params]))

    # Estimate energies
    job = estimator.run(circuits)
    results = job.result()
    energies = [res.data.evs[0] for res in results]

    # Return the total energie (cost function)
    if len(weights) != len(energies):
        raise ValueError("Length of weights must match number of basis states.")
    return np.dot(weights, energies)


In [4]:
def opti_loop_ssvqe(cost_func, x0, basis_state_indices, hamiltonian, estimator, num_qubits, method="COBYLA", weights=None, shared_ansatz=None):
    return minimize(
        cost_func,
        x0,
        args=(basis_state_indices, hamiltonian, estimator, num_qubits, weights, shared_ansatz),
        method=method
    )


General function to run the SSVQE algorithm :

In [5]:
def run_ssvqe(
    hamiltonian: SparsePauliOp,
    num_qubits: int,
    num_states: int,
    estimator,
    weights: list[float],
    ansatz_template: QuantumCircuit = None,
    method: str = "COBYLA"
) -> tuple[np.ndarray, list[Statevector]]:
    """
    Runs SSVQE and returns the lowest `num_states` eigenvalues and statevectors.

    Returns:
      eigenvalues: array of shape (num_states,)
      statevectors: list of Statevector length num_states
    """
    # 1) Build default ansatz if none provided
    if ansatz_template is None:
        ansatz_template = TwoLocal(
            num_qubits=num_qubits,
            rotation_blocks=["ry","rz"],
            entanglement_blocks="cx",
            entanglement="full",
            reps=1,
            parameter_prefix="θ"
        )

    # 2) Initial parameters & reference indices
    num_params = len(ansatz_template.parameters)
    x0 = np.random.uniform(0, 2*np.pi, size=num_params)
    basis_state_indices = list(range(num_states))

    # 3) Optimize the weighted cost
    result = opti_loop_ssvqe(
        cost_func_ssvqe,
        x0,
        basis_state_indices,
        hamiltonian,
        estimator,
        num_qubits,
        method=method,
        weights=weights,
        shared_ansatz=ansatz_template
    )
    optimal_params = result.x

    # 4) Build the final ansatz with those parameters
    param_dict = dict(zip(ansatz_template.parameters, optimal_params))
    fixed_ansatz = ansatz_template.assign_parameters(param_dict)

    # 5) Prepare each eigenstate and measure its energy
    eigenvalues = []
    statevectors = []
    for idx in basis_state_indices:
        basis_circ = basis_state_circuit(idx, num_qubits)
        full_circ  = basis_circ.compose(fixed_ansatz)
        sv         = Statevector(full_circ)
        energy     = sv.expectation_value(hamiltonian).real
        eigenvalues.append(energy)
        statevectors.append(sv)

    # 6) Sort by energy
    pairs = sorted(zip(eigenvalues, statevectors), key=lambda p: p[0])
    ev_sorted, sv_sorted = zip(*pairs)

    return np.array(ev_sorted), list(sv_sorted)


### Test Case 1 — 2-Qubit Non-degenerate Hamiltonian

**Hamiltonian**  
\begin{equation*}
H = Z\otimes Z \;+\; 2\,X\otimes X
\end{equation*}

**Spectrum**  
-3, -1, +1, +3

**Eigenstates**  
- $\displaystyle \ket{\psi_0} = \frac{1}{\sqrt2}\bigl(\ket{01} - \ket{10}\bigr)$  
- $\displaystyle \ket{\psi_1} = \frac{1}{\sqrt2}\bigl(\ket{00} - \ket{11}\bigr)$  
- $\displaystyle \ket{\psi_2} = \frac{1}{\sqrt2}\bigl(\ket{01} + \ket{10}\bigr)$  
- $\displaystyle \ket{\psi_3} = \frac{1}{\sqrt2}\bigl(\ket{00} + \ket{11}\bigr)$



In [6]:
# 1) Define Hamiltonian
obs1 = SparsePauliOp.from_list([
    ("ZZ", 1.0),
    ("XX", 2.0)
])

# 2) SSVQE parameters
num_qubits = 2
k = 4
weights = [32,16,8,4]

# 3) Shared ansatz template (applied to each reference state)
ansatz1 = TwoLocal(
    num_qubits=num_qubits,
    rotation_blocks=["ry","rz"],
    entanglement_blocks="cx",
    entanglement="full",
    reps=1,
    parameter_prefix="θ"
)

# 4) Run SSVQE
eigenvals_ssvqe, states_ssvqe = run_ssvqe(
    hamiltonian=obs1,
    num_qubits=num_qubits,
    num_states=k,
    estimator=estimator,
    weights=weights,
    ansatz_template=ansatz1,
    method="COBYLA"
)

# 5) Sort and print results
pairs_ssvqe = sorted(zip(eigenvals_ssvqe, states_ssvqe), key=lambda x: x[0])
threshold = 1e-3

def fix_global_phase(vec, threshold = 1e-3):
    v = vec.copy()
    # find first significant index
    for amp in v:
        if abs(amp) > threshold:
            phi = np.angle(amp)
            v *= np.exp(-1j * phi)
            if v[np.argmax(np.abs(v))].real < 0:
                v *= -1
            break
    return v

for ev, sv in pairs_ssvqe:
    vec = fix_global_phase(sv.data)
    terms = [
        f"({amp:.5f})|{format(idx, f'0{num_qubits}b')}⟩"
        for idx, amp in enumerate(vec)
        if abs(amp) > threshold
    ]
    print(f"λ = {ev:.6f} → " + " + ".join(terms))


λ = -3.000000 → (0.70713-0.00000j)|01⟩ + (-0.70709+0.00004j)|10⟩
λ = -1.000000 → (0.70713-0.00000j)|00⟩ + (-0.70709-0.00005j)|11⟩
λ = 1.000000 → (0.70709-0.00000j)|01⟩ + (0.70713-0.00004j)|10⟩
λ = 3.000000 → (0.70709-0.00000j)|00⟩ + (0.70713+0.00005j)|11⟩


### Test Case 2 — 3-Qubit Mixed-Interaction Hamiltonian using the TwoLocal ansatz

**Hamiltonian**  
\begin{equation*}
H = X\otimes X\otimes Z \;+\; Y\otimes Y\otimes Z \;+\; Z\otimes Z\otimes Z
\end{equation*}

**Spectrum (degenerate)**
-3, -1, +1, +3


**Eigenstates (only the first 4)**
\begin{align*}
\lambda_0 &= -3, & \ket{\psi_0} &= \tfrac{1}{\sqrt2}\bigl(\ket{010} - \ket{100}\bigr),\\
\lambda_1 &= -1, & \ket{\psi_1} &= \ket{001},\\
\lambda_2 &= -1, & \ket{\psi_2} &= \tfrac{1}{\sqrt2}\bigl(\ket{011} + \ket{101}\bigr),\\
\lambda_3 &= -1, & \ket{\psi_3} &= \ket{111}.
\end{align*}



In [7]:
# 1) Hamiltonian
obs2 = SparsePauliOp.from_list([
    ("XXZ", 1.0),
    ("YYZ", 1.0),
    ("ZZZ", 1.0)
])


# 2) SSVQE parameters
num_qubits = 3

# In fact we compute the 5 first eigenvalues, otherwise it struggles to find
# the 3 eigenvectors related to lambda=-1
k = 5

weights = [64,32,16,8,4]

# 3) Shared ansatz template (applied to each reference state)
ansatz2 = TwoLocal(
    num_qubits=num_qubits,
    rotation_blocks=["ry","rz"],
    entanglement_blocks="cx",
    entanglement="full",
    reps=2
)

# 4) Run SSVQE
eigenvals_ssvqe, states_ssvqe = run_ssvqe(
    hamiltonian=obs2,
    num_qubits=num_qubits,
    num_states=k,
    estimator=estimator,
    weights=weights,
    ansatz_template=ansatz2,
    method="COBYLA"
)

# 5) Sort and print results
pairs_ssvqe = sorted(zip(eigenvals_ssvqe, states_ssvqe), key=lambda x: x[0])
threshold = 1e-3


for ev, sv in pairs_ssvqe:
    vec = fix_global_phase(sv.data)
    terms = [
        f"({amp:.5f})|{format(idx, f'0{num_qubits}b')}⟩"
        for idx, amp in enumerate(vec)
        if abs(amp) > threshold
    ]
    print(f"λ = {ev:.6f} → " + " + ".join(terms))
    

λ = -3.000000 → (-0.70709+0.00000j)|010⟩ + (0.70712-0.00004j)|100⟩
λ = -1.000000 → (-0.66054+0.00000j)|001⟩ + (0.21849+0.12621j)|011⟩ + (0.21850+0.12620j)|101⟩ + (0.33006+0.57220j)|111⟩
λ = -1.000000 → (0.70712+0.00000j)|001⟩ + (0.35330+0.61250j)|111⟩
λ = -1.000000 → (0.25233+0.00000j)|001⟩ + (0.57197+0.33039j)|011⟩ + (0.57200+0.33040j)|101⟩ + (-0.12607-0.21857j)|111⟩
λ = 1.000000 → (0.25233+0.00000j)|000⟩ + (0.57199+0.33042j)|010⟩ + (0.57198+0.33037j)|100⟩ + (-0.12607-0.21857j)|110⟩


In the presence of degeneracy (here, three eigenvalues equal to −1), VQD will return some basis of that degenerate subspace—but not necessarily the exact eigenvectors define before. To verify that our three VQD-computed “excited” states truly span the correct λ = −1 subspace, we use the **projector test**:

1. **Exact projector**  
   \begin{equation*}
     P_{\rm exact}
     = \sum_{j:\,\lambda_j=-1} \ket{\psi_j}\!\bra{\psi_j}
     = \Psi\,\Psi^\dagger
   \end{equation*}
   built from the three known eigenvectors at λ = −1.

2. **VQD projector**  
   \begin{equation*}
     P_{\rm VQD}
     = \sum_{j=1}^3 \ket{\phi_j}\!\bra{\phi_j}
     = \Phi\,\Phi^\dagger
   \end{equation*}
   built from the three VQD-computed states.

3. **Frobenius-norm check**  
   \begin{equation*}
     \delta = \|P_{\rm exact} - P_{\rm VQD}\|_F.
   \end{equation*}
   If $\delta \approx 0$ (up to numerical noise), the two projectors—and therefore the two 3-dimensional subspaces—are identical.  

This test is **basis-independent** and gives a single answer about whether VQD captured the correct subspace.  


In [8]:
# 1) Exact λ = −1 subspace (3 vectors in 8-dim Hilbert space)
#    Basis: 000=0,001=1,010=2,011=3,100=4,101=5,110=6,111=7
psi = np.zeros((8, 3), complex)
psi[1,0] = 1.0                                  # |001⟩
psi[3,1] = 1/np.sqrt(2); psi[5,1] = 1/np.sqrt(2) # (|011⟩+|101⟩)/√2
psi[7,2] = 1.0                                  # |111⟩

# 2) Extract your VQD-computed states at λ≈−1
phi = np.column_stack([states_ssvqe[i].data for i in [1, 2, 3]])  # shape (8,3)
phi = np.column_stack([fix_global_phase(col) for col in phi.T])

# 3) Build projectors
P_exact = psi @ psi.conj().T
P_vqd   = phi @ phi.conj().T

# 4) Frobenius-norm of their difference
delta = np.linalg.norm(P_exact - P_vqd, ord='fro')
print(f"Projector difference ‖P_exact – P_vqd‖ₚ = {delta:.6f}")


Projector difference ‖P_exact – P_vqd‖ₚ = 0.000174


### Test case 3 - Random 3-Qubit Hamiltonian

1. **Generate** a random 3-qubit Hermitian Hamiltonian $H$.  
2. **Diagonally** solve it exactly (ground + first three excited states) using SciPy.  
3. **Variationally** approximate the same four eigenpairs with SSVQE.  
4. **Compare** the classical and variational results side by side.


In [9]:
def random_hermitian_matrix(n):
    dim = 2**n
    A = 3*(np.random.rand(dim, dim) + 1j*np.random.rand(dim, dim))
    H = (A + A.conj().T) / 2  # Make it Hermitian
    return H

n=3
H_matrix = random_hermitian_matrix(n)

We use `scipy.linalg.eigh` to obtain all eigenvalues/vectors, then display the lowest four.

In [10]:
from scipy.linalg import eigh

# Diagonalize
eigenvals, eigenvecs = eigh(H_matrix)

# Take the first four eigenpairs
first4_vals = eigenvals[:4]
first4_vecs = eigenvecs[:, :4]  

# Print each eigenvalue and its statevector
threshold = 1e-3
for val, vec in zip(first4_vals, first4_vecs.T):
    vec_fixed = fix_global_phase(vec, threshold)
    terms = [
        f"({amp:.5f})|{format(idx, f'0{int(np.log2(len(vec_fixed)))}b')}⟩"
        for idx, amp in enumerate(vec_fixed)
        if abs(amp) > threshold
    ]
    print(f"λ = {val:.6f} → " + " + ".join(terms))


λ = -4.431614 → (0.16620-0.00000j)|000⟩ + (-0.20355-0.21691j)|001⟩ + (0.31918+0.47078j)|010⟩ + (-0.07883-0.42298j)|011⟩ + (-0.14076-0.00339j)|100⟩ + (0.13737-0.42377j)|101⟩ + (-0.13922+0.22084j)|110⟩ + (-0.09519+0.28243j)|111⟩
λ = -2.900531 → (0.14344+0.00000j)|000⟩ + (0.06062-0.05128j)|001⟩ + (-0.24699-0.07523j)|010⟩ + (0.11051-0.09425j)|011⟩ + (0.38508+0.41102j)|100⟩ + (-0.21436-0.27580j)|101⟩ + (-0.06351+0.52512j)|110⟩ + (-0.13326-0.38545j)|111⟩
λ = -1.065875 → (0.48339-0.00000j)|000⟩ + (-0.15568+0.13681j)|001⟩ + (-0.11984-0.39693j)|010⟩ + (-0.17256-0.41788j)|011⟩ + (0.07812-0.13936j)|100⟩ + (0.16973+0.41784j)|101⟩ + (-0.25975+0.16557j)|110⟩ + (-0.01030+0.15217j)|111⟩
λ = -0.040035 → (-0.26253+0.00000j)|000⟩ + (-0.26894-0.03567j)|001⟩ + (0.25860+0.11804j)|010⟩ + (-0.34050-0.12565j)|011⟩ + (0.36694+0.22181j)|100⟩ + (0.43379+0.19455j)|101⟩ + (-0.01550-0.20995j)|110⟩ + (-0.05229-0.43361j)|111⟩


SSVQE Approximation :


In [11]:
k = 4
weights = [32, 16, 8, 4]

# 1. Build the 4-qubit TwoLocal ansatz
ansatz3 = TwoLocal(
    num_qubits=n,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=3
)

# 2. Hamiltonian from random H_matrix
obs3 = SparsePauliOp.from_operator(Operator(H_matrix))

# 3. Run SSVQE
eigenvals3, states3 = run_ssvqe(
    hamiltonian=obs3,
    num_qubits=num_qubits,
    num_states=k,
    estimator=estimator,
    weights=weights,
    ansatz_template=ansatz3,
    method="COBYLA"
)

# 4. Sort eigenvalues and corresponding states
pairs3 = sorted(zip(eigenvals3, states3), key=lambda x: x[0])


# 5. Print each eigenvalue and its significant components
threshold = 1e-3
for ev, st in pairs3:
    vec = fix_global_phase(st.data, threshold)
    terms = [
        f"({amp:.5f})|{format(idx, f'0{int(np.log2(len(vec_fixed)))}b')}⟩"
        for idx, amp in enumerate(vec)
        if abs(amp) > threshold
    ]
    print(f"λ = {ev:.6f} →", " + ".join(terms))

λ = -4.303749 → (0.21805-0.00000j)|000⟩ + (-0.33552-0.08186j)|001⟩ + (0.42380+0.26794j)|010⟩ + (-0.21824-0.37675j)|011⟩ + (-0.13031+0.05966j)|100⟩ + (-0.05006-0.48345j)|101⟩ + (-0.06408+0.22803j)|110⟩ + (0.00771+0.28157j)|111⟩
λ = -2.113773 → (0.38154-0.00000j)|000⟩ + (-0.04344-0.00974j)|001⟩ + (-0.14683-0.06267j)|010⟩ + (-0.04970-0.28216j)|011⟩ + (0.45537-0.36013j)|100⟩ + (-0.22047+0.32733j)|101⟩ + (0.15659+0.28297j)|110⟩ + (-0.35734+0.14065j)|111⟩
λ = -0.247268 → (-0.15943+0.00000j)|000⟩ + (-0.39630-0.08882j)|001⟩ + (0.31433+0.48747j)|010⟩ + (0.22786+0.16379j)|011⟩ + (0.06549-0.20684j)|100⟩ + (-0.30864+0.19725j)|101⟩ + (0.24572-0.23644j)|110⟩ + (-0.03366-0.30953j)|111⟩
λ = 0.965294 → (-0.23193+0.00000j)|000⟩ + (0.09330-0.00709j)|001⟩ + (0.11651-0.20193j)|010⟩ + (0.47976+0.27230j)|011⟩ + (-0.10670-0.49148j)|100⟩ + (-0.20116-0.27532j)|101⟩ + (-0.10039+0.33963j)|110⟩ + (0.01723+0.28960j)|111⟩


Finally, we print the first four eigenpairs side-by-side, with the 2-norm 
between each exact and variational state to quantify the error.

In [12]:
print("Comparison of first 4 eigenpairs (Classical vs VQD):\n")
for i in range(4):
    # Classical
    cl_val = first4_vals[i]
    cl_vec = fix_global_phase(first4_vecs[:, i])
    # VQD (sorted results in pairs3)
    vq_val, vq_state = pairs3[i]
    vq_vec = fix_global_phase(vq_state.data)
    
    # Norm difference
    diff = np.linalg.norm(cl_vec - vq_vec)
    
    print(f"Eigenpair {i}:")
    print(f"  Classical λ = {cl_val:.6f}")
    print(f"  VQD       λ = {vq_val:.6f}")
    print(f"  ‖φ_cl – φ_vqd‖₂ = {diff:.6f}\n")

Comparison of first 4 eigenpairs (Classical vs VQD):

Eigenpair 0:
  Classical λ = -4.431614
  VQD       λ = -4.303749
  ‖φ_cl – φ_vqd‖₂ = 0.413605

Eigenpair 1:
  Classical λ = -2.900531
  VQD       λ = -2.113773
  ‖φ_cl – φ_vqd‖₂ = 1.240122

Eigenpair 2:
  Classical λ = -1.065875
  VQD       λ = -0.247268
  ‖φ_cl – φ_vqd‖₂ = 1.704229

Eigenpair 3:
  Classical λ = -0.040035
  VQD       λ = 0.965294
  ‖φ_cl – φ_vqd‖₂ = 1.811320

